In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [3]:
!pip install texthero
!pip install rouge

     |████████████████████████████████| 235 kB 6.1 MB/s 
     |████████████████████████████████| 1.5 MB 29.6 MB/s 
     |████████████████████████████████| 749 kB 34.9 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [4]:
import texthero as hero
from rouge import Rouge
from string import digits, punctuation
import torch
import time
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import string 
import os
import re
import queue
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
df_more = pd.read_csv('/content/drive/MyDrive/LSTM/news_summary_more.csv')
df_more.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [6]:
df_more['head_len'] = df_more['headlines'].apply(lambda x : len(x.split(' ')))
df_more['text_len'] = df_more['text'].apply(lambda x : len(x.split(' ')))

In [7]:
df_more['head_len'].max(), df_more['text_len'].max()

(18, 92)

In [8]:
df_more.drop(['head_len', 'text_len'], axis=1, inplace=True)
df_more.columns = ['highlights', 'stories']
df = df_more.copy(deep=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device to run the code on GPU

In [9]:
df['stories'] = df['stories'].astype(str)
df['highlights'] = df['highlights'].astype(str)

remove_digits = str.maketrans('', '', digits)
df['stories'] = df['stories'].apply(lambda x : x.translate(remove_digits))
df['highlights'] = df['highlights'].apply(lambda x : x.translate(remove_digits))

df['stories_len'] = df['stories'].apply(lambda x : len(x.split(' ')))
df['highlights_len'] = df['highlights'].apply(lambda x : len(x.split(' ')))
df.head()

,highlights,stories,stories_len,highlights_len
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",60,13
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...,60,12
2,New Zealand end Rohit Sharma-led India's -matc...,New Zealand defeated India by wickets in the ...,60,9
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer...",60,9
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...,60,13


In [10]:
max_sentence_length = 60
df = df[df['stories_len'] <= max_sentence_length]
df.shape

(91208, 4)

In [11]:
df['stories'] = hero.clean(df['stories'])
df['highlights'] = hero.clean(df['highlights'])

df['stories'] = df['stories'].apply(lambda x : 'sos ' + x + ' eos')
df['highlights'] = df['highlights'].apply(lambda x : 'sos ' + x + ' eos')

In [12]:
df.head()

,highlights,stories,stories_len,highlights_len
0,sos upgrad learner switches career ml al salar...,sos saurav kant alumnus upgrad iiit b pg progr...,60,13
1,sos delhi techie wins free food swiggy one yea...,sos kunal shah credit card bill payment platfo...,60,12
2,sos new zealand end rohit sharma led india mat...,sos new zealand defeated india wickets fourth ...,60,9
3,sos aegon life iterm insurance plan helps cust...,sos aegon life iterm insurance plan customers ...,60,9
4,sos known hirani yrs metoo claims true sonam eos,sos speaking sexual harassment allegations raj...,60,13


In [13]:
stories_ = df.iloc[:,1].values
highlights_ = df.iloc[:,0].values
output_len =  df.iloc[:,3].values

In [13]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-12-08 22:35:17--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-12-08 22:35:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-12-08 22:35:17--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [14]:
embedding_dict = dict()

f = open('/content/drive/MyDrive/LSTM/glove.6B.300d.txt', encoding = 'utf8')

for line in f:
    values = line.split()
    word = values[0]
    coef = np.asarray(values[1:],dtype='float32')
    embedding_dict[word] = coef 
f.close()

In [15]:
# create vocab
words = [] 

for line in stories_:
    for word in line.split(' '):
        words.append(word)

for line in highlights_:
    for word in line.split(' '):
        words.append(word)

In [16]:
vocab = sorted(Counter(words), key=Counter(words).get, reverse=True)

In [17]:
embed_dim = 300
words_found = 0
words_OOV = 0
word2idx = {}
word2idx['<pad>'] = 0

for index, word in enumerate(vocab):
    word2idx[word] = index+1

weight_embedding = np.zeros([len(vocab)+1,embed_dim],dtype='float32')

In [18]:
for i, word in enumerate(vocab):
    try:
        weight_embedding[i,:] = embedding_dict[word]
        words_found += 1
    except:
        weight_embedding[i,:] = np.random.normal(scale=0.6, size=[embed_dim,])
words_found        

634

In [19]:
weight_embedding = torch.from_numpy(weight_embedding)

In [20]:
# to integers

stories_num = []
highlights_num = []

for w in stories_:
    sentence = []
    for word in w.split():
        sentence.extend([word2idx[word]])
    stories_num.append(sentence)

for w in highlights_:
    sentence = []
    for word in w.split():
        sentence.extend([word2idx[word]])
    highlights_num.append(sentence)

In [21]:
max_story_len = 40 
max_summary_len = 40
story_feature = np.zeros((len(stories_num),max_story_len),dtype=int)
highlight_feature = np.zeros((len(highlights_num),max_summary_len), dtype=int)


for i, row in enumerate(stories_num):
    story_feature[i,:len(row)] = np.array(row)[:max_story_len]
for i, row in enumerate(highlights_num):
    highlight_feature[i,:len(row)] = np.array(row)[:max_summary_len]

In [22]:
# train test split
split_index = int(len(story_feature)*0.8)
train_story, test_story = story_feature[:split_index],story_feature[split_index:]
train_highlight, test_highlight = highlight_feature[:split_index],highlight_feature[split_index:]

In [23]:
split_index = int(len(test_story)*0.1)
val_story, test_story  = test_story[:split_index],test_story[split_index:]
val_highlight, test_highlight  = test_highlight[:split_index],test_highlight[split_index:]

In [24]:
val_story.shape, test_story.shape, train_story.shape

((1824, 40), (16418, 40), (72966, 40))

In [25]:
def get_batches(x, y,batch_size=16):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [26]:
def create_emb(weight_matrix, non_trainable=False):
    emb_layer = torch.nn.Embedding(weight_matrix.shape[0],weight_matrix.shape[1])
    emb_layer.load_state_dict({'weight': weight_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False 
    return emb_layer

In [27]:
class Encoder(nn.Module):
    
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, weight_matrix):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = create_emb(weight_matrix,True)

        #self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

    def forward(self, x):
        embedding = self.dropout(self.embedding(x))
        print(embedding.shape)
        outputs, (hidden, cell) = self.rnn(embedding)
        return hidden, cell

In [28]:
class Decoder(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p, weight_matrix):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = create_emb(weight_matrix,True)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))

        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)

        return predictions, hidden, cell

In [29]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(vocab)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        hidden, cell = self.encoder(source)

        x = target[0]

        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            outputs[t] = output
            best_guess = output.argmax(1)

            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

In [30]:
load_model = False
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(vocab)
input_size_decoder = len(vocab)
output_size = len(vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300

hidden_size = 256
num_layers = 1
enc_dropout = 0.2
dec_dropout = 0.2

In [31]:
encoder_net = Encoder(input_size_encoder, 
                      encoder_embedding_size,
                      hidden_size,num_layers, 
                      enc_dropout, weight_embedding).to(device)


decoder_net = Decoder(input_size_decoder, 
                      decoder_embedding_size,
                      hidden_size,output_size,num_layers, 
                      dec_dropout, weight_embedding).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1



In [32]:
import random

In [ ]:
epochs = 5
batch_size = 128
learning_rate = 0.001
model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index = 0)

counter = 0
total_loss = []

for e in range(epochs):
    start_time = time.time() # start timer
    p = np.random.permutation(train_highlight.shape[0])
    train_story = train_story[p,:]
    train_highlight = train_highlight[p,:]
    
    epoch_loss = 0

    for x,y in get_batches(train_story,train_highlight,batch_size):
        
        model.train()
        x = torch.from_numpy(x).to(device)
        y = torch.from_numpy(y).to(device)
        
        output  = model(x, y)
        output = output[1:].reshape(-1, output.shape[2])
        y = y[1:].reshape(-1)
        optimizer.zero_grad()
        loss = criterion(output, y)

        epoch_loss += loss
        # Back prop
        loss.backward()

        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()
        
        print("Epoch: {}/{} \tStep: {} \tLoss: {:.4f} ".format(e+1, epochs, counter, loss))
        counter += 1
        
    total_loss.append(epoch_loss)
    torch.save(model.state_dict(), '/content/drive/MyDrive/LSTM/Vanilla_weight.pth')

Streaming output truncated to the last 5000 lines.
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 268 	Loss: 7.0306 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 269 	Loss: 6.9170 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 270 	Loss: 6.9820 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 271 	Loss: 6.9149 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 272 	Loss: 7.0291 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 273 	Loss: 6.9511 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 274 	Loss: 7.1293 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 275 	Loss: 6.9818 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 276 	Loss: 7.0178 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 277 	Loss: 7.0036 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 278 	Loss: 6.9285 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 279 	Loss: 6.9511 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 280 	Loss: 7.0086 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 281 	Loss: 6.9251 
torch.Size([128, 40, 300])
Epoch: 1/5 	Step: 282 	Los

In [33]:
num_hidden = 256 
enc_layers = 1 
batch_size = 32
vocab_size = len(word2idx) 
dec_layers = 1 
embed_dims = 300 

In [34]:
# loading the weights of the pre-trained model
model = Seq2Seq(encoder_net, decoder_net).to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/LSTM/Vanilla_weight.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

### ROUGE SCORE

In [35]:
# predicting the output
import random
result = []

with torch.no_grad():
    beam_time = time.time() # start timer
    loss_beam = [] 
    beam_predict = [] # save beam search decoder outputs
    highlight_validation = [] # save validation target summaries
    story_validation = [] # save validation input text
    model.eval()
    # initialize the encoder hidden state

    for x_val, y_val in get_batches(val_story[:batch_size*57], val_highlight[:batch_size*57],batch_size):
        # convert data to PyTorch tensor
        x_val = torch.from_numpy(x_val).to(device).long()
        y_val = torch.from_numpy(y_val).to(device).long()
        # run the beam search decoder
        temp = model.forward(source=x_val, target=y_val)
        temp = torch.argmax(temp, dim=2)
        result.append(temp)
    model.train()

torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([32, 40, 300])
torch.Size([

In [38]:
res = np.array(result[0].cpu())

for i in range(1, len(result)):
  res = np.append(res, np.array(result[i].cpu()), axis=0)
res.shape

(1824, 40)

In [39]:
idx2word = dict((v,k) for k,v in word2idx.items())

In [40]:
def concatenate__reference(refer):

  op = refer[: batch_size, :]

  for i in range(1, 57):
    op = np.append(op, refer[batch_size*i : batch_size*(i+1), :], axis=0)
  return op

reference = concatenate__reference(val_highlight)
reference.shape, type(reference)

((1824, 40), numpy.ndarray)

In [44]:
# converting index to word 
def numpy_to_str(array_):

  result = []
  for i in range(array_.shape[0]):

    temp = ""
    for j in range(array_.shape[1]):

      try:
        temp += idx2word[array_[i][j]] + ' '
      except:
        pass
    result.append(temp)
  return result
  
reference_ = numpy_to_str(reference)
model_out = numpy_to_str(res)

In [49]:
# calculates rouge score
rouge = Rouge()
scores = rouge.get_scores(reference_, model_out, avg=True)

In [50]:
scores

{'rouge-1': {'f': 0.2986931395652834,
  'p': 0.20362965494544416,
  'r': 0.5938687865497024},
 'rouge-2': {'f': 0.008132339948672698,
  'p': 0.004914806806254171,
  'r': 0.03529070071010861},
 'rouge-l': {'f': 0.29862004015007876,
  'p': 0.20357483038404067,
  'r': 0.5937591374268953}}